<a href="https://colab.research.google.com/github/Abhineswari/Fake_document_detection/blob/main/Fake_document_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#pip install langchain

In [ ]:
import os
import getpass
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_API_KEY'] = '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'

In [ ]:
#pip install -qU langchain-openai

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

 ········


In [ ]:
#pip install langchain_chroma


In [ ]:
#pip install langchainhub


In [ ]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
rag_chain.invoke("What is Task Decomposition?")

'Task decomposition is a technique that breaks down complex tasks into smaller and simpler steps. It allows agents to better understand and plan for each component of a task. Methods like Chain of Thought and Tree of Thoughts help in decomposing tasks for easier management and interpretation.'

In [ ]:
!pip install pytesseract
!apt-get install -y tesseract-ocr
!pip install Pillow
!pip install transformers


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 29 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (4,646 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 124947 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [ ]:
from google.colab import files
from PIL import Image
import pytesseract

# Upload the PNG document
uploaded = files.upload()

# Open the uploaded image
img_path = next(iter(uploaded))  # Get the first uploaded file
image = Image.open(img_path)
image.show()  # Display the image


In [ ]:
# Extract text using pytesseract OCR
extracted_text = pytesseract.image_to_string(image)
print("Extracted Text:\n", extracted_text)


Extracted Text:
 Bahawasanya atas nama Yang di-Pertuan Agong Malaysia
diminta semua yang berkenaan supaya membenarkan
pembawa pasport ini lalu dengan bebas tanpa halangan
atau sekatan dan memberi apa-apa jua pertolongan dan

perlindungan yang perlu kepadanya. 5

These are to request and require in the name of the Supreme |Z
‘Head of Malaysia all those whom it may concern to allow the oe

or eae Siecenoeri one Na

nN

io
K

MALAYSIA

Jeni / Type Kod Negara / Country Code No. Pasport/ Passport
- MYS A15640175
Nama / Name
IZADDIN BIN ARIS

No. Pengenalan / identty No.

Warganegara / Natonaity
MALAYSIA 511110025431

Tempat Lahir/ Place of Birth Tarikh Lahir/ Date of Birth
KEDAH 10 NOV 1951

Jantina | Sex Tinggi Height

LELAKI-M 168 cm

Tarikh diveluarkan / Date of issue Tarkh Tamat Sahiaku / Date of Expiry
13 APR 2008 13 APR 2013

Pejabat Pengeluar / Issuing Office

KUALA LUMPUR

P<MYSIZADDIN<BIN<ARIS<<<<<<<<<<<<cececececececececececccece
A15641755MYS5111109M1104134511110025431<<<<<<<<<<<<

In [ ]:
# Simple fake or real classification based on missing clauses
def classify_document(text):
    # List of important legal terms that should be present
    required_keywords = ['Indemnification', 'Termination', 'Governing Law', 'Confidentiality']

    missing_keywords = [keyword for keyword in required_keywords if keyword.lower() not in text.lower()]

    if len(missing_keywords) > 0:
        return f"Fake Document - Missing key clauses: {', '.join(missing_keywords)}"
    else:
        return "Real Document - Contains all necessary clauses."

# Classify the uploaded document
classification_result = classify_document(extracted_text)
print("\nDocument Classification Result:", classification_result)



Document Classification Result: Fake Document - Missing key clauses: Indemnification, Termination, Governing Law, Confidentiality


In [ ]:
from transformers import pipeline

# Load a pre-trained transformer model for classification (for fake/real detection)
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Define candidate labels for classification
candidate_labels = ["fake", "real"]

# Classify the document as real or fake based on the extracted text
result = classifier(extracted_text, candidate_labels)
print("\nFake/Real Classification Result:", result)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu



Fake/Real Classification Result: {'sequence': 'Bahawasanya atas nama Yang di-Pertuan Agong Malaysia\ndiminta semua yang berkenaan supaya membenarkan\npembawa pasport ini lalu dengan bebas tanpa halangan\natau sekatan dan memberi apa-apa jua pertolongan dan\n\nperlindungan yang perlu kepadanya. 5\n\nThese are to request and require in the name of the Supreme |Z\n‘Head of Malaysia all those whom it may concern to allow the oe\n\nor eae Siecenoeri one Na\n\nnN\n\nio\nK\n\nMALAYSIA\n\nJeni / Type Kod Negara / Country Code No. Pasport/ Passport\n- MYS A15640175\nNama / Name\nIZADDIN BIN ARIS\n\nNo. Pengenalan / identty No.\n\nWarganegara / Natonaity\nMALAYSIA 511110025431\n\nTempat Lahir/ Place of Birth Tarikh Lahir/ Date of Birth\nKEDAH 10 NOV 1951\n\nJantina | Sex Tinggi Height\n\nLELAKI-M 168 cm\n\nTarikh diveluarkan / Date of issue Tarkh Tamat Sahiaku / Date of Expiry\n13 APR 2008 13 APR 2013\n\nPejabat Pengeluar / Issuing Office\n\nKUALA LUMPUR\n\nP<MYSIZADDIN<BIN<ARIS<<<<<<<<<<<<cece